# Advanced options

In the previous lesson we learned about the basic and intermediate options of Tensor Fox. For most of applications this is enough, but sometimes one needs to change more parameters, add constraints, and so on. In this lesson we also cover the options regarding higher order tensors. Warning: this lesson has a more mathematical flavour.

Options already covered:

    display
    maxiter  
    tol     
    tol_step
    tol_improv
    tol_grad
    tol_mlsvd
    trunc_dims
    initialization
    refine    
    init_damp
    symm    
    tol_jump
    
Options to be covered:

    method
    inner_method 
    cg_maxiter 
    cg_factor
    cg_tol 
    constraints 
    trials 
    bi_method
    bi_method_maxiter 
    bi_method_tol 
    epochs 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import TensorFox as tfx
from IPython.display import Image

class options:    # start creating an empty class
    pass

options = tfx.make_options(options)    # update the class

In [2]:
# Create the tensor.
m = 2
T = np.zeros((m, m, m))
s = 0

for k in range(m):
    for i in range(m):
        for j in range(m):
            T[i,j,k] = s
            s += 1

# Inner algorithm options

The method we are using to solve the problem of tensor approximation is called *damped Gauss-Newton* (dGN), and at each step of this method the program needs to solve a equation of the form

$$(J^T J + \mu D) x = J^Tb$$
as already mentioned. To solve this equation we have to rely on other method, which can be an iterative method like the [conjugate gradient](https://en.wikipedia.org/wiki/Conjugate_gradient_method) (default) or a direct method using matrix factorization. The conjugate gradient methods has its own parameters, and the user may have to tune them sometimes. With this in mind, Tensor Fox offers the parameters $\verb|inner| \_ \verb|method|, \ \verb|cg| \_ \verb|maxiter|, \ \verb|cg| \_ \verb|tol|$ and $\verb|cg| \_ \verb|factor|$. They are explained below.

The inner methods are: $\verb|cg|, \ \verb|cg| \_ \verb|static|$, $\verb|direct|$ and $\verb|als|$ (alternating least squares, but this one doesn't take in account the regularization). We also mention that it is possible to pass the parameter  $\verb|inner| \_ \verb|method|$ as a list of strings containing the names of the method available. Then the program uses the prescribed sequence of methods, one at each iterarion. We noticed that this hybrid way of work can bring good results sometimes. 

The difference between static and non-static versions are the way the program deals with the maximum number of iterations. The static algorithm have a certain maximum number of iterations $\verb|cg| \_ \verb|maxiter|$ which is fixed during all the program. The non-static versions uses the parameter $\verb|cg| \_ \verb|factor|$ to control the number of iterations in a different way. If the program is at the $k$-th iteration of the dGN, then the maximum number of iterations permitted for the cg method is
 
$$1 + int\left( \verb|cg|\_\verb|factor| \cdot \verb|randint|\left( 1 + k^{0.4}, 2 + k^{0.9} \right) \right).$$

This strange interval of random integers were obtained after a lot of tests, a lot! This seems to be a robust choice, but since we can't be right all the time, the parameter $\verb|cg| \_ \verb|factor|$ comes to the rescue. If the number of maximum iterations are increasing too much, just set this parameter to a low value such as $0.1$ or $0.5$. Finally, the parameter $\verb|cg| \_ \verb|tol|$, as the name suggests, is the tolerance parameter for the cg method. The cg iterations stops when the (absolute) residual is less than $\verb|cg| \_ \verb|tol|$. Below there is an example showing how to setup a method and its parameters.

In [3]:
# Let's use cg_static as the inner algorithm, with 3 iterations max and tolerance of 1e-7.
options.inner_method = 'cg_static'
options.cg_maxiter = 3
options.cg_tol = 1e-7
options.display = 2

R = 3
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    No compression detected
    Working with dimensions (2, 2, 2)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
    Iteration | Rel error |  Step size  | Improvement | norm(grad) | Predicted error | # Inner iterations
        1     | 5.76e+00  |  7.24e+00   |  5.76e+00   |  4.80e+00  |    1.23e+00     |        3        
        2     | 5.65e+00  |  6.01e-03   |  1.12e-01   |  1.12e+03  |    4.61e-03     |        3        
        3     | 5.34e+00  |  1.74e-02   |  3.07e-01   |  1.09e+03  |    3.26e-02     |        3        
        4     | 4.62e+00  |  4.86e-02   |  7.26e-01   |  1.01e+03  |

       126    | 3.92e-02  |  8.77e-04   |  1.05e-04   |  3.71e-01  |    2.96e-07     |        3        
       127    | 3.91e-02  |  8.32e-04   |  1.41e-04   |  4.34e-01  |    2.52e-07     |        3        
       128    | 3.90e-02  |  1.04e-03   |  8.82e-05   |  3.88e-01  |    3.44e-07     |        3        
       129    | 3.88e-02  |  7.88e-04   |  1.80e-04   |  4.54e-01  |    2.28e-07     |        3        
       130    | 3.87e-02  |  1.05e-03   |  9.03e-05   |  3.82e-01  |    3.71e-07     |        3        
       131    | 3.86e-02  |  8.64e-04   |  1.81e-04   |  4.65e-01  |    2.42e-07     |        3        
       132    | 3.84e-02  |  1.01e-03   |  1.11e-04   |  3.84e-01  |    3.54e-07     |        3        
       133    | 3.83e-02  |  9.17e-04   |  1.78e-04   |  4.60e-01  |    2.67e-07     |        3        
       134    | 3.82e-02  |  1.11e-03   |  1.06e-04   |  3.94e-01  |    3.91e-07     |        3        
       135    | 3.80e-02  |  9.22e-04   |  2.05e-04   |  4.75e-0

# Constraints

The parameter $\verb|factors| \_ \verb|norm|$ is used to fix the norm of the factor matrices of the CPD. Suppose $T$ is a third tensor and $(X^{(k)}, Y^{(k)}, Z^{(k)})$ the approximated CPD at iteration $k$. If one set $\verb|factors| \_ \verb|norm| = 2$, for example, then $\| X^{(k)} \| = \| Y^{(k)} \| = \| Z^{(k)} \| = 2$ for all $k$.

# Higher order tensors and the Tensor Train format

Tensor Fox has distinct approaches when it comes to computing the CPD of third order tensors and higher order tensors. By default the program relies on the *Damped Gauss-Newton* (dGN) method. However you can set the program to use the *Tensor Train format* (TT format), also called *Tensor Train decomposition*. Without going in too much details, we use a specific configuration of the TT format which can be obtained by computing several third order CPD's. More precisely, if $T$ is a tensor of order $L$, then we can compute a CPD for it by computing $L-2$ third order CPD's. Once we have the TT format of $T$, the CPD can also be computed. The figure below illustrate the representation of a tensor train associated to a tensor of order $L$.

![tensortrain](tensor-train.png)

Each square represent the coordinates of a tensor and each circle is the coordinate with is shared between two consecutive tensors. These tensors are usually denoted by $\mathcal{G}^{(\ell)}$. For example, the second tensor is $\mathcal{G}^{(2)}$, which has coordinates $\mathcal{G}^{(\ell)}_{j_1 i_2 j_2}$, and the next tensor is $\mathcal{G}^{(3)}$, which has coordinates $\mathcal{G}^{(\ell)}_{j_2 i_3 j_3}$. The first and last tensor are acutally matrices (which mean $j_0 = j_L = 1$), and the other $L-2$ tensor are third order tensors. They are related to $T$ by the following formula:
$$T_{i_1 i_2 \ldots i_L} = \sum_{j_0, j_1, \ldots, j_L} \mathcal{G}^{(1)}_{j_0 i_1 j_1} \cdot \mathcal{G}^{(2)}_{j_1 i_2 j_2} \cdot \ldots \cdot \mathcal{G}^{(L)}_{j_{L-1} i_L j_L}.$$

By computing a CPD for $\mathcal{G}^{(2)}, \ldots, \mathcal{G}^{(L-1)}$ we can obtain a CPD for $T$.

## Trials

In the case $T$ has order higher than $3$, the parameter $\verb|trials|$ defines how much times we compute each one of these third order CPD's. The idea is to compute several times and keep the best result (smaller error). This may be helpful because all $L-2$ CPD's needs to be of good quality in order to get a good CPD for $T$. If just one of the third order CPD's has bad precision, than everything falls apart. Currently the default is $\verb|trials|= 3$, but this may change depending on the problem. This parameter doesn't makes difference if $T$ is a third order tensor. 

## Display

As we've said, the options $\verb|trials|$ says about the repetition of third order CPD computations. If $\verb|display|$ is set to $1, 2, 3$ or $4$, then all the information of each one of these CPD's are printed on the screen. This means we wil have $(L-2) \cdot \verb|trials|$ CPD's informations printed on the screen when $T$ has order $L$ . Sometimes this amount of information is just too much. We can make everything more succint in these situations just by setting $\verb|display| =-1$. Consider the following fourth order tensor.

In [4]:
# Initialize dimensions of the tensor.
k = 2
dims = (k+1, k+1, k+1, k+1)
L = len(dims)

# Create four random factors matrices so that
# A = (orig_factors[0], orig_factors[1], orig_factors[2], orig_factors[3])*I.
orig_factors = []
for l in range(L):
    M = np.random.randn(dims[l], k)
    Q, R = np.linalg.qr(M)
    orig_factors.append(Q)
    
# From the factor matrices generate the respective tensor in coordinates.
A = tfx.cpd2tens(orig_factors)

print('A = ')
tfx.showtens(A) # now this is the same as print(A)

A = 
[[[[ 0.34508452 -0.51591871 -0.25320509]
   [ 0.13590675 -0.17943987 -0.09242195]
   [-0.02323685  0.05551406  0.0234353 ]]

  [[ 0.10382666 -0.13578541 -0.07020707]
   [ 0.00458415 -0.07253009 -0.02355082]
   [-0.03875144 -0.00752367  0.00831344]]

  [[ 0.02613346 -0.02127416 -0.01370515]
   [-0.022944   -0.03056259 -0.00310266]
   [-0.03083407 -0.01797356  0.00293041]]]


 [[[ 0.26377359 -0.33621695 -0.17567334]
   [-0.0046921  -0.19260784 -0.05791587]
   [-0.11274105 -0.03001607  0.02168855]]

  [[-0.00952183 -0.15043532 -0.04362881]
   [ 0.30378221  0.1620054  -0.03350389]
   [ 0.26966306  0.20367603 -0.01133965]]

  [[-0.06139204 -0.07057153 -0.00485751]
   [ 0.27903004  0.18676917 -0.01910489]
   [ 0.26937347  0.19244647 -0.01471191]]]


 [[[-0.17703548  0.31545684  0.14550782]
   [-0.1645573   0.04362525  0.05853243]
   [-0.07103783 -0.0917602  -0.00872537]]

  [[-0.13090024  0.02891955  0.0447832 ]
   [ 0.25992321  0.22713502 -0.00145823]
   [ 0.28128658  0.18678164 -0.019

In [5]:
# Compute the CPD of A with succint display for higher order tensors.
class options:    
    pass

options = tfx.make_options(options)    
options.display = -1
options.method = 'ttcpd'
    
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)

Total of 2 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 8.67445000718831e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 7.97218214573518e-16

Final results
    Number of steps = 203
    Relative error = 8.233920416410934e-16
    Accuracy =  100.0 %


In [6]:
# The options display = -2 is showed below. 
options.display = -2
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)
    Compression relative error = 1.392816e-15

SVD Tensor train error =  7.867157439336931e-16

Total of 2 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 7.450187984020279e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 2.57551701886408e-15

CPD Tensor train error =  3.7822835975232603

Final results
    Number of steps = 88
    Relative error = 3.0111745697723974e-15
    Accuracy =  100.0 %


## MLSVD tolerance with high order tensors

Now let's see what happens when we set $\verb|tol| \_ \verb|mlsvd| = [\verb|1e-6|, \verb|-1|]$ for the tensor $A$. This choice means the program will perform the high order compression using $10^{-6}$ as tolerance, and will not compress the intermediate third order tensors. 

In [7]:
options.tol_mlsvd = [1e-6, -1]
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (3, 3, 3, 3) to (2, 2, 2, 2)
    Compression relative error = 7.651926e-16

SVD Tensor train error =  1.2707935837471802e-15

Total of 2 third order CPDs to be computed:
CPD 1 error = 1.6332216545761085e-16
CPD 2 error = 7.011373747739042e-16

CPD Tensor train error =  6.1965725360990875

Final results
    Number of steps = 172
    Relative error = 1.0872956717743662e-15
    Accuracy =  100.0 %


## Inner algorithm options

Just as usual third order tensors has the options $\verb|method|, \verb|tol|, \verb|maxiter|$ for its inner computations, the third order tensors of a tensor train also can receive these parameters. However there is a difference here: when computing the CPD's of each $\mathcal{G}^{(\ell)}$, the program starts computing the CPD of $\mathcal{G}^{(2)}$, and one factor is of the CPD is used to compute the CPD of $\mathcal{G}^{(3)}$. Then one factor of this CPD is used to compute the CPD of $\mathcal{G}^{(4)}$ and so on. In short, each CPD depends on the previous computed CPD. The matrices $\mathcal{G}^{(1)}$ and $\mathcal{G}^{(L)}$ are easily computed after we have the other CPD's.

The first CPD can be computed as any CPD, but the other always depends on some previous computed factor, which is always used to fix one factor of the next CPD. This means each CPD, except the first, is actually only computing two factors, so there is a difference in how the program computes the first CPD and the remaining ones. Therefore, the parameters $\verb|method|, \ \verb|method| \_ \verb|tol|, \ \verb|method| \_ \verb|maxiter|$ are used for the first CPD and the parameters $\verb|bi| \_ \verb|method|, \ \verb|bi| \_ \verb|method| \_ \verb|tol|, \ \verb|bi| \_ \verb|method| \_ \verb|maxiter|$ are used for all the remaining CPD's. The figure below illustrate these observations.

![tensortrainmethods](tensor-train-methods.png)

## Epochs

As we can note, there is a flow of information in the tensor train format, the CPD's are computed from left to right, and the next CPD always depend on some information about the previous CPD. Once we compute the CPD of $\mathcal{G}^{(L-1)}$ it is possible to "go back", that is, use the information of the CPD of $\mathcal{G}^{(L-1)}$ to compute a new CPD for $\mathcal{G}^{(L-2)}$, we just have to reverse the way information is propagated. Doing this we may be able to refine all CPD's. These cycles can repeated several times, with the information being propagated forward and backward again and again. Each cycle is called an *epoch*, and the number of epochs can be passed to the program through the parameter $\verb|epochs|$. Below we redefine the tensor $A$ to have a higher order and try to refine the CPD by using more epochs than just $1$.

In [8]:
# Initialize the sixth-order tensor and compute its CPD with default options.
k = 2
dims = (k+1, k+1, k+1, k+1, k+1, k+1)
L = len(dims)

orig_factors = []
for l in range(L):
    M = np.random.randn(dims[l], k)
    Q, R = np.linalg.qr(M)
    orig_factors.append(Q)
    
A = tfx.cpd2tens(orig_factors)

class options:    
    pass

options = tfx.make_options(options)    
options.display = -1
options.method = 'ttcpd'
    
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compressing unfolding mode 5
    Compressing unfolding mode 6
    Compression detected
    Compressing from (3, 3, 3, 3, 3, 3) to (2, 2, 2, 2, 2, 2)

Total of 4 third order CPDs to be computed:
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 3.407713739362484e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 6.86279447989679e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 3 error = 6.411064843351227e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 4 error = 9.986223135919356e-16

Final results
    

In [9]:
# Now we use 5 epochs on the same tensor.
options.epochs = 5
options.display = -1
factors, output = tfx.cpd(A, k, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compressing unfolding mode 5
    Compressing unfolding mode 6
    Compression detected
    Compressing from (3, 3, 3, 3, 3, 3) to (2, 2, 2, 2, 2, 2)

Total of 4 third order CPDs to be computed:
Epoch  1
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 1 error = 8.475550751948159e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 2 error = 1.422099079035713e-15
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 3 error = 7.139309208754597e-16
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
CPD 4 error = 4.0839384567343154e-16

Epoch  

# Method

The last parameter to be seen is $\verb|method|$. By default Tensor Fox uses $\verb|method| = \verb|'dGN'|$, which means the program will use the damped Gauss-Newton method. For higher order tensors, $\verb|method| = \verb|'ttcpd'|$ can be a good choice when the rank is smaller than all dimensions. This method is recommended for dense tensors since it avoids the curse of dimensionality. We note that it is also possible to set $\verb|method| = \verb|'ttcpd'|$ for third order tensors. Finally, another possibility is $\verb|method| = \verb|'als'|$ (Alternating Least Squares), a classic and well known method to compute CPDs.